In [4]:
import pandas as pd
import datetime as dt
import numpy as np
import requests
import json
from dateutil.parser import *
import matplotlib.pyplot as plt
import time
from dateutil import tz
import pickle
import pytz
import plotly.graph_objects as go
from notify_run import Notify 

In [5]:
def pushbullet_message(title, body = None):
    msg = {"type": "note", "title": title, "body": body}
    TOKEN = 'x'
    resp = requests.post('https://api.pushbullet.com/v2/pushes', 
                         data=json.dumps(msg),
                         headers={'Authorization': 'Bearer ' + TOKEN,
                                  'Content-Type': 'application/json'})
    if resp.status_code != 200:
        raise Exception('Error',resp.status_code)
    else:
        print ('Message sent')

#pushbullet_message('process completed')

In [6]:
API_KEY = 'x'
ACCOUNT_ID = 'x'
OANDA_URL = 'https://api-fxpractice.oanda.com/v3'

SECURE_HEADER = {
    'Authorization': f'Bearer {API_KEY}',
    'Content-Type': 'application/json'
}

In [42]:
data_library = {
    'GBP_JPY' :  { "pair" :  'GBP_JPY', "units": 3000, "pip": 0.01},
    'GBP_AUD' :  { "pair" :  'GBP_AUD', "units": 3000, "pip": 0.0001},
    'GBP_USD' :  { "pair" :  'GBP_USD', "units": 3000, "pip": 0.0001},
    'USD_CAD' :  { "pair" :  'USD_CAD', "units": 3000,"pip": 0.0001},
    'USD_JPY' :  { "pair" :  'USD_JPY', "units": 3000, "pip": 0.01},
    'AUD_JPY' :  { "pair" :  'AUD_JPY', "units": 3000,"pip": 0.01},
    'AUD_CAD' :  { "pair" :  'AUD_CAD', "units": 3000, "pip": 0.0001},
    'EUR_USD' :  { "pair" :  'EUR_USD', "units": 3000, "pip": 0.0001},
    'EUR_JPY' :  { "pair" :  'EUR_JPY', "units": 3000, "pip": 0.01},
    'EUR_AUD' :  { "pair" :  'EUR_AUD', "units": 3000, "pip": 0.0001},
    'CAD_JPY' :  { "pair" :  'CAD_JPY', "units": 3000, "pip": 0.01},
    'AUD_USD' :  { "pair" :  'AUD_USD', "units": 3000, "pip": 0.0001},
    'XAU_USD' :  { "pair" :  'XAU_USD', "units": 3,    "pip": 0.1},
    'NZD_USD' :  { "pair" :  'NZD_USD', "units": 3000, "pip": 0.0001},
    'AUD_NZD' :  { "pair" :  'AUD_NZD', "units": 3000, "pip": 0.0001},
    'EUR_NZD' :  { "pair" :  'EUR_NZD', "units": 3000, "pip": 0.0001},
    'NZD_JPY' :  { "pair" :  'NZD_JPY', "units": 3000, "pip": 0.01},
    'GBP_NZD' :  { "pair" :  'GBP_NZD', "units": 3000, "pip": 0.0001},
    'NZD_CAD' :  { "pair" :  'NZD_CAD', "units": 3000, "pip": 0.0001},
    'EUR_CAD' :  { "pair" :  'EUR_CAD', "units": 3000, "pip": 0.0001},
    'EUR_GBP' :  { "pair" :  'EUR_GBP', "units": 3000, "pip": 0.0001},
    'GBP_CAD' :  { "pair" :  'GBP_CAD', "units": 3000, "pip": 0.0001},
    'USD_CHF' :  { "pair" :  'USD_CHF', "units": 3000, "pip": 0.0001},
    'BCO_USD' :  { "pair" :  'BCO_USD', "units": 30, "pip": 0.1},
    'CORN_USD' :  { "pair" :  'CORN_USD', "units": 3, "pip": 0.01},
    'XCU_USD' :  { "pair" :  'XCU_USD', "units": 3, "pip": 0.0001},
    'NATGAS_USD' :  { "pair" :  'NATGAS_USD', "units": 3, "pip": 0.01},
    'XAG_USD' :  { "pair" :  'XAG_USD', "units": 3, "pip": 0.0001},
    'WTICO_USD' :  { "pair" :  'WTICO_USD', "units": 30, "pip": 0.1},
    'XPD_USD' :  { "pair" :  'XPD_USD', "units": 3, "pip": 0.01},
    'XPT_USD' :  { "pair" :  'XPT_USD', "units": 3, "pip": 0.01},
    'SOYBN_USD' :  { "pair" :  'SOYBN_USD', "units": 3, "pip": 0.01},
    'SUGAR_USD' :  { "pair" :  'SUGAR_USD', "units": 3, "pip": 0.0001},
    'WHEAT_USD' :  { "pair" :  'WHEAT_USD', "units": 3, "pip": 0.01}

    }

In [7]:
#getting the pkl file of certain pair
def get_his_data_filename(pair, granularity):
    return f"downloaded data/{pair}_{granularity}.pkl"

def get_results_filename(pair, granularity):
    return f"Backtest Results/{pair}_{granularity}_break_results.pkl"

def parser(row):
    d= parse(row)
    return d

def astimezone(row):
    return row.astimezone(pytz.timezone('Singapore'))


In [8]:
#oanda time has time zone. so we get utc time and add the tz at the back with replace
def time_utc():
    return dt.datetime.utcnow().replace(tzinfo=dt.timezone.utc)

#convert any date string with dateutil and then put a tz at the back
def get_utc_dt_from_string(date_str):
    d = parse(date_str)
    return d.replace(tzinfo=dt.timezone.utc)

#print(dt.datetime.utcnow()) -> this no timezone
#print(time_utc()) -> this has timezone
#print(get_utc_dt_from_string("2020-02-01 03:00:00"))

In [15]:
class OandaAPI():
    #any oandaapi object that queries object.session will be calling requests.session
    def __init__(self):
        self.session = requests.Session()    
    
    #get instruments dict
    def fetch_instruments(self):
        url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"
        response = self.session.get(url, params=None, headers=SECURE_HEADER)
        return response.status_code, response.json()
    
    #get instruments as df
    def get_instruments_df(self):
        code, data = self.fetch_instruments()
        if code == 200:
            df = pd.DataFrame.from_dict(data['instruments'])
            return df[['name', 'type', 'displayName', 'pipLocation', 'marginRate']]
        else:
            return None
        
    #save instruments
    def save_instruments(self):
        df = self.get_instruments_df()
        if df is not None:
            df.to_pickle(get_instruments_data_filename())
    
    #get currency candles data from server, default count is none, granularity is 1h
    def fetch_candles(self, pair_name, count=None, granularity="H1", date_from=None, date_to=None, as_df=False):
        url = f"{OANDA_URL}/instruments/{pair_name}/candles"

        params = dict(
            granularity = granularity,
            price = "MBA"
        )
        
        if date_from is not None and date_to is not None:
            params['to'] = int(date_to.timestamp())
            params['from'] = int(date_from.timestamp())
        elif count is not None:
            params['count'] = count
        else:
            params['count'] = 300
            
        try:
            response = self.session.get(url, params=params, headers=SECURE_HEADER)
        
        except:
            print('GET error, sleep')
            time.sleep(3)
            response = self.session.get(url, params=params, headers=SECURE_HEADER)
            print (response.status_code)
            
        if response.status_code != 200:
            return response.status_code, None

        if as_df == True:
            try:
                json_data = response.json()['candles']
                return response.status_code, OandaAPI.candles_to_df(json_data)
            except:
                print('sleep')
                time.sleep(3)
                response = self.session.get(url, params=params, headers=SECURE_HEADER)
                json_data = response.json()['candles']
                print (response.status_code)
                return response.status_code, OandaAPI.candles_to_df(json_data)
        else:        
            return response.status_code, response.json()

    @classmethod
    def candles_to_df(cls, json_data):
        prices = ['mid', 'bid', 'ask']
        ohlc = ['o', 'h', 'l', 'c']

        our_data = []
        for candle in json_data:
            if candle['complete'] == False:
                continue
            new_dict = {}
            new_dict['time'] = candle['time']
            new_dict['volume'] = candle['volume']
            for price in prices:
                for oh in ohlc:
                    new_dict[f"{price}_{oh}"] = float(candle[price][oh])
            our_data.append(new_dict)
        df = pd.DataFrame.from_dict(our_data)
        return df

#if __name__ == "__main__":
    #api = OandaAPI()
    #api.save_instruments()
    #date_from = get_utc_dt_from_string("2019-05-05 18:00:00")
    #date_to = get_utc_dt_from_string("2019-05-10 18:00:00")
    #res, df = api.fetch_candles("EUR_USD", date_from=date_from, date_to=date_to, as_df=True)
    #print(df.head())

In [17]:
INCREMENTS = {
    'M1':1
    #'M5' : 5,
    #'H1' : 60,
    #H4' : 240
}

#input time
def create_file(pair, granularity, api):
    candle_count = 2000
    time_step = INCREMENTS[granularity] * candle_count

    date_from = get_utc_dt_from_string("2018-01-01 23:59:59")
    #time_utc()- dt.timedelta(46)
    #get_utc_dt_from_string("2020-01-01 23:59:59")
    end_date = get_utc_dt_from_string("2021-06-21 23:59:59")
    #time_utc()
    candle_dfs = []
    #get_utc_dt_from_string("2021-06-21 23:59:59")

    date_to = date_from
    #change date_to to whatever date_from is plus the time delta. after first loop is done, our date_to >
    # will be updated again and it takes the next chunk
    while date_to < end_date:
        date_to = date_from + dt.timedelta(minutes=time_step)
        if date_to > end_date:
            date_to = end_date
                
        code, df = api.fetch_candles(pair,
                granularity=granularity,
                date_from=date_from,
                date_to=date_to,as_df=True)
        if df is not None and df.empty ==False:
            candle_dfs.append(df)

        elif code != 200:
            print("ERROR", pair, granularity, date_from, date_to)
            break
        date_from = date_to
    #put all the chunks of data into the list into a df
    final_df = pd.concat(candle_dfs)
    final_df.drop_duplicates(subset='time', inplace=True) 
    final_df.sort_values(by='time', inplace=True)
    final_df.to_pickle(get_his_data_filename(pair, granularity))
    print(f"{pair} {granularity} {final_df.iloc[0].time}  {final_df.iloc[-1].time}")

In [18]:
# Input to obtain data from site
pair = ['BCO_USD','CORN_USD','XCU_USD', 'NATGAS_USD', 'XAG_USD', 'WTICO_USD', 'XPD_USD', 'XPT_USD', 'SOYBN_USD','SUGAR_USD', 'WHEAT_USD']
#['GBP_JPY','GBP_USD','EUR_JPY','EUR_USD','USD_CAD','XAU_USD','AUD_JPY','AUD_CAD','EUR_AUD','GBP_AUD','USD_JPY','CAD_JPY', 'AUD_USD']
#['NZD_USD','AUD_NZD','EUR_NZD', 'NZD_JPY', 'GBP_NZD', 'NZD_CAD', 'EUR_CAD', 'EUR_GBP', 'GBP_CAD', 'XAU_GBP', 'XAU_EUR', 'USD_CHF']
#['BCO_USD','CORN_USD','XCU_USD', 'NATGAS_USD', 'XAG_USD', 'WTICO_USD', 'XPD_USD', 'XPT_USD', 'SOYBN_USD','SUGAR_USD', 'WHEAT_USD']
pair_list = pair

api = OandaAPI()
for g in INCREMENTS.keys():
    for pair in pair_list:
        create_file(pair, g, api)
#Notify().send('completed')

#pd.read_pickle(get_his_data_filename(pair,'M1'))

BCO_USD M1 2018-01-02T01:00:00.000000000Z  2021-06-21T21:59:00.000000000Z
CORN_USD M1 2018-01-02T14:30:00.000000000Z  2021-06-21T18:19:00.000000000Z
XCU_USD M1 2018-01-02T00:00:00.000000000Z  2021-06-21T23:59:00.000000000Z
NATGAS_USD M1 2018-01-02T00:00:00.000000000Z  2021-06-21T23:54:00.000000000Z
XAG_USD M1 2018-01-01T23:59:00.000000000Z  2021-06-21T23:59:00.000000000Z
WTICO_USD M1 2018-01-02T00:00:00.000000000Z  2021-06-21T23:59:00.000000000Z
XPD_USD M1 2018-01-01T23:59:00.000000000Z  2021-06-21T23:45:00.000000000Z
XPT_USD M1 2018-01-02T00:01:00.000000000Z  2021-06-21T23:59:00.000000000Z
SOYBN_USD M1 2018-01-02T14:30:00.000000000Z  2021-06-21T18:19:00.000000000Z
SUGAR_USD M1 2018-01-02T10:14:00.000000000Z  2021-06-21T16:59:00.000000000Z
WHEAT_USD M1 2018-01-02T14:30:00.000000000Z  2021-06-21T18:19:00.000000000Z


In [26]:
# Upload data into historical file

df2 = pd.read_pickle('/Users/User2/Desktop/project/A Sure Bot/downloaded data/2018-210621_M1_Data_new_150821.pkl')

pairlist = ['BCO_USD','CORN_USD','XCU_USD', 'NATGAS_USD', 'XAG_USD', 'WTICO_USD', 'XPD_USD', 'XPT_USD', 'SOYBN_USD','SUGAR_USD', 'WHEAT_USD']
for pair in pairlist:
    df1 = pd.read_pickle(get_his_data_filename(pair,'M1'))
    df1.reset_index(inplace=True,drop=True)
    df1['time'] = df1['time'].apply(parser)
    df1['PAIR']= pair
    #type(df1['time'][0])
    df1.to_pickle(get_his_data_filename(pair,'M1'))

    df2=df2.append(df1)
    df2.reset_index(inplace=True,drop=True)
    df2.to_pickle('/Users/User2/Desktop/project/A Sure Bot/downloaded data/2018-210621_M1_Data_new_150821.pkl')


In [27]:
def get_res_sup(temp_df,candles):
    global pivots_resist
    global pivots_support
    global pivot_dates_support
    global pivot_dates_resist
    global date_detected_support
    global date_detected_resist
    
    pivots_resist =[]
    pivots_support =[]
    pivot_dates_support =[]
    pivot_dates_resist = []
    date_detected_support = []
    date_detected_resist = []
    value_range = [0]*candles
    value_range2 = [100000]*candles
    date_range = [0]*candles    
    max_counter =0
    min_counter = 0

    for i in temp_df.index:

        max_value = max(value_range)
        min_value = min(value_range2)
    
        value_range = value_range[1:candles]
        value_range.append(round(temp_df['mid_c'][i],4))
    
        value_range2 = value_range2[1:candles]
        value_range2.append(round(temp_df['mid_c'][i],4))
    
        date_range=date_range[1:candles]
        date_range.append(temp_df['time'][i])

        if max_value==max(value_range):
            max_counter+=1
        
        if max_value<max(value_range):
            max_counter =0
     
        if min_value==min(value_range2):
            min_counter+=1
         
        if min_value>min(value_range2):
            min_counter =0

        if max_counter ==(candles/2):
            date_index = value_range.index(max_value)
            date_pivot = date_range[date_index]
            if date_pivot>temp_df['time'][0]+dt.timedelta(5):
                pivots_resist.append(max_value)
                pivot_dates_resist.append(date_pivot)
                date_detected_resist.append(temp_df['time'][i])

        if min_counter ==(candles/2):
            date_index = value_range2.index(min_value)
            date_pivot = date_range[date_index]
            if date_pivot>temp_df['time'][0]+dt.timedelta(5):
                pivots_support.append(min_value)   
                pivot_dates_support.append(date_pivot)
                date_detected_support.append(temp_df['time'][i])
 
    for i in [0,-1]:
        print ('resist: '+str(pivots_resist[i])+ ' : '+ str(pivot_dates_resist[i]) + ' : ' + str(date_detected_resist[i]))
        print ('support: '+str(pivots_support[i])+ ' : '+ str(pivot_dates_support[i]) + ' : ' + str(date_detected_support[i]))

    df_support = pd.DataFrame({'dates': pivot_dates_support, 'sure_level': pivots_support, 'date_detected': date_detected_support})
    df_resist = pd.DataFrame({'dates': pivot_dates_resist, 'sure_level': pivots_resist, 'date_detected': date_detected_resist})
    df_support.to_pickle(f'/Users/User2/Desktop/project/A Sure Bot/Backtest sure_levels/{pair}_pivots_support_{candles}_010118_210621.pkl')
    df_resist.to_pickle(f'/Users/User2/Desktop/project/A Sure Bot/Backtest sure_levels/{pair}_pivots_resist_{candles}_010118_210621.pkl')

In [39]:
#break support function

#candles = 2880
#pivot_support_df = pd.read_pickle(f'Backtest sure_levels/{pair}_pivots_support_{candles}_010118_210621.pkl')

#pips_exceed_point=[5]
#pips_tp_point = [30]
#pips_sl_point = [20]

#pips_exceed = [data_library[pair]['pip']*x for x in pips_exceed_point]
#pips_tp = [data_library[pair]['pip']*x for x in pips_tp_point]
#pips_sl = [data_library[pair]['pip']*x for x in pips_sl_point]
#delta_time = [46]
#sure_delta = '24h'

def break_support(df,pips_exceed,pips_tp,delta_time,sure_delta,pips_sl):
 break_support_suc_count = 0
 break_support_loss_count = 0

 break_support_columns = ['buy_date','sell_date','PAIR','sure_level','decision','delta_time','pips_exceed','pips_tp', 'upper_limit','lower_limit',
                            'suc_price','fail_price','pre-entry-candle','break_candle','prev_close_m30','sure_date','sure_detected','sure_delta']
 
 break_support_df =pd.DataFrame(columns = break_support_columns)

 df.reset_index(inplace=True,drop=True)
    
 for p in pips_exceed:
    for r in pips_tp:
     for t in delta_time:
        for x in range(len(pivot_support_df)):
            price1= pivot_support_df['sure_level'][x]
            upper_limit = price1+p
            lower_limit = price1-p
            Date1 = pivot_support_df['date_detected'][x]
            df['dates_to_end'] = (df['time'] > Date1)
            
            for i in range(len(df.index)):
                if df['dates_to_end'][i]==True:
                    if df['bid_l'][i]<=(lower_limit) and df['prev_bid_l'][i]>(lower_limit): 
                        if df['prev_close_m30'][i]>df['mid_c'][i]:
                            
                            break_buy_date= df['time'][i]
                            break_candle = df['bid_l'][i]

                            df['Dates after bought']= (df['time']>break_buy_date) & (df['time']<=Date1+dt.timedelta(t))
                
                            for y in range(len(df.index)):
                                
                                if df['Dates after bought'][y] == True:
                            
                                    if df['ask_l'][y]<=(lower_limit-r):
                                    
                                        break_support_suc_count +=1
                                        break_support_df = break_support_df.append(pd.Series([break_buy_date,df['time'][y],pair,price1,-1,
                                                                                            t,p/data_library[pair]['pip'],'100/50',
                                                                                            upper_limit,lower_limit, df['ask_l'][y],np.nan,
                                                                                            df['prev_bid_l'][i],break_candle,df['prev_close_m30'][i],
                                                                                            pivot_support_df['dates'][x], Date1, sure_delta],
                                                                                            index=break_support_columns), ignore_index=True)

                                        break
                                        
                                    if df['ask_h'][y]>=(lower_limit+pips_sl):

                                        break_support_loss_count +=1
                                        break_support_df = break_support_df.append(pd.Series([break_buy_date,df['time'][y],pair,price1,-1,
                                                                                            t,p/data_library[pair]['pip'],'100/50',
                                                                                            upper_limit,lower_limit, np.nan, df['ask_h'][y],
                                                                                            df['prev_bid_l'][i],break_candle,df['prev_close_m30'][i],
                                                                                            pivot_support_df['dates'][x],Date1,sure_delta],
                                                                                            index=break_support_columns),ignore_index=True)

                                        break
                            break
 print(break_support_suc_count/(break_support_suc_count+break_support_loss_count))
 return break_support_df

#break_support_df = break_support(df,pips_exceed,pips_tp,delta_time,sure_delta)     

#display(break_support_df.count())
#display(break_support_df)
#break_support_df.to_pickle(f'Backtest Results/{pair}_break_spread.pkl')
#break_support_df.to_pickle(f'Backtest Results/{pair}_break_tp30.pkl')
#Notify().send('completed')

#Onboarding of new pair - add backtested data to log
def append_break_sup(break_support_df):
    temp_df = break_support_df
    temp_df.reset_index(inplace=True,drop=True)
    temp_df['sure_date']=temp_df['sure_date'].apply(astimezone)
    
    main_df = pd.read_pickle('break_stop_log2.pkl')
    main_df.reset_index(inplace=True,drop=True)

    for i in temp_df.index:
        new_df = pd.DataFrame({'time':[temp_df['sure_date'][i]], 
                                'PAIR': [temp_df['PAIR'][i]],
                                'sure_level': [temp_df['sure_level'][i]],
                                'decision': [temp_df['decision'][i]]
                                })
        main_df = main_df.append(new_df)
    return main_df

df2= append_break_sup(break_support_df)
display(df2)

#df2.to_pickle('break_stop_log2.pkl')

In [40]:
#break resist function

#candles = 2880
#pivot_resist_df = pd.read_pickle(f'Backtest sure_levels/{pair}_pivots_resist_{candles}_010118_210621.pkl')

#pips_exceed_point=[5]
#pips_tp_point = [30]
#pips_sl_point = [20]

#pips_exceed = [data_library[pair]['pip']*x for x in pips_exceed_point]
#pips_tp = [data_library[pair]['pip']*x for x in pips_tp_point]
#pips_sl = [data_library[pair]['pip']*x for x in pips_sl_point]
#delta_time = [46]
#sure_delta = '24h'

def break_resist(df,pips_exceed,pips_tp,delta_time,sure_delta,pips_sl):
    break_resist_suc_count = 0
    break_resist_loss_count = 0

    break_resist_columns = ['buy_date','sell_date','PAIR','sure_level','decision','delta_time','pips_exceed','pips_tp', 'upper_limit','lower_limit',
                            'suc_price','fail_price','pre-entry-candle','break_candle','prev_close_m30','sure_date','sure_detected','sure_delta']
 
    break_resist_df =pd.DataFrame(columns = break_resist_columns)

    df.reset_index(inplace=True,drop=True)
    
    for p in pips_exceed:
        for r in pips_tp:
            for t in delta_time:
                for x in range(len(pivot_resist_df)):
                    price1= pivot_resist_df['sure_level'][x]
                    upper_limit = price1+p
                    lower_limit = price1-p
                    Date1 = pivot_resist_df['date_detected'][x]
                    df['dates_to_end'] = (df['time'] > Date1)
                    
                    for i in range(len(df.index)):
                        if df['dates_to_end'][i]==True:
                            if df['ask_h'][i]>=(upper_limit) and df['prev_ask_h'][i]<(upper_limit): 
                                if df['prev_close_m30'][i]<df['mid_c'][i]:
                                
                                    break_buy_date= df['time'][i]
                                    break_candle = df['ask_h'][i]
                                    df['Dates after bought']= (df['time']>break_buy_date) & (df['time']<=Date1+dt.timedelta(t))
                    
                                    for y in range(len(df.index)):
                                    
                                        if df['Dates after bought'][y] == True:
                                
                                            if df['bid_h'][y]>=(upper_limit+r):
                                        
                                                break_resist_suc_count +=1
                                                break_resist_df = break_resist_df.append(pd.Series([break_buy_date,df['time'][y],pair,price1,1,
                                                                                                    t,p/data_library[pair]['pip'],'100/50',
                                                                                                    upper_limit,lower_limit, df['bid_h'][y],np.nan,
                                                                                                    df['prev_ask_h'][i],break_candle,df['prev_close_m30'][i],
                                                                                                    pivot_resist_df['dates'][x],Date1,sure_delta],
                                                                                                    index=break_resist_columns),ignore_index=True)
                                                
                                                break
                                            
                                            if df['bid_l'][y]<=(upper_limit-pips_sl):

                                                break_resist_loss_count +=1
                                                break_resist_df = break_resist_df.append(pd.Series([break_buy_date,df['time'][y],pair,price1,1,
                                                                                                    t,p/data_library[pair]['pip'],'100/50',
                                                                                                    upper_limit,lower_limit, np.nan,df['bid_l'][y],
                                                                                                    df['prev_ask_h'][i],break_candle,df['prev_close_m30'][i],
                                                                                                    pivot_resist_df['dates'][x],Date1,sure_delta],
                                                                                                    index=break_resist_columns),ignore_index=True)
                                                break
                                    break

    print(break_resist_suc_count/(break_resist_suc_count+break_resist_loss_count))
    
    return break_resist_df

#break_resist_df = break_resist(df,pips_exceed,pips_tp,delta_time,sure_delta)     

#print(break_resist_df.count())
#display(break_resist_df)
#Notify().send('completed') 

#appending break resist backtest results to file saved by break above support

#df1 = pd.read_pickle(f'Backtest Results/{pair}_break_spread_tp30.pkl')
#df1=df1.append(break_resist_df)
#df1.sort_values(['buy_date'],inplace=True)
#df1.drop_duplicates(subset = ['sure_date'],keep = 'last',inplace=True)
#df1.reset_index(inplace=True,drop=True)
#df1
#df1.to_pickle(f'Backtest Results/{pair}_break_spread_tp30.pkl')
#pd.read_pickle(f'Backtest Results/{pair}_break_spread_tp30.pkl')


#Onboarding of new pair - add backtested data to log
def append_break_res(break_resist_df):
    temp_df = break_resist_df
    temp_df.reset_index(inplace=True,drop=True)
    temp_df['sure_date']=temp_df['sure_date'].apply(astimezone)
    
    main_df = pd.read_pickle('break_stop_log2.pkl')
    main_df.reset_index(inplace=True,drop=True)

    for i in temp_df.index:
        new_df = pd.DataFrame({'time':[temp_df['sure_date'][i]], 
                                'PAIR': [temp_df['PAIR'][i]],
                                'sure_level': [temp_df['sure_level'][i]],
                                'decision': [temp_df['decision'][i]]
                                })
        main_df = main_df.append(new_df)
    return main_df

df2= append_break_res(break_resist_df)
display(df2)

#df2.to_pickle('break_stop_log2.pkl')

In [56]:
#can apply this before so save it into a df and dont need to keep reloading
hist_df = pd.read_pickle('/Users/User2/Desktop/project/A Sure Bot/downloaded data/2015-2017_M1_Data_new_150821.pkl')

In [57]:
#Load and save sure levels to file/ creating backtest results file
pair_list = ['BCO_USD','WTICO_USD', 'XCU_USD']
# ['BCO_USD','CORN_USD','XCU_USD', 'NATGAS_USD', 'XAG_USD', 'WTICO_USD', 'XPD_USD', 'XPT_USD', 'SOYBN_USD','SUGAR_USD', 'WHEAT_USD']
#['AUD_CAD','EUR_AUD','GBP_AUD', 'GBP_CAD', 'EUR_CAD', 'NZD_CAD', 'GBP_NZD','NZD_JPY','AUD_NZD','EUR_NZD']

for pair in pair_list:
    granularity = 'M1'

    #option1 - read from loaded save file above
    #df = pd.read_pickle(get_his_data_filename(pair, granularity))

    #option2 - read from historical data
    #df2 = pd.read_pickle('downloaded data/01012018-21062021_M1_Data.pkl') -> take out of cell so that dont need to keep running
    df = hist_df[hist_df['PAIR']== pair].copy()

    non_cols = ['time','volume','PAIR']
    mod_cols = [x for x in df.columns if x not in non_cols]
    df[mod_cols]=df[mod_cols].apply(pd.to_numeric)
    df.reset_index(inplace=True,drop=True)

    #df['time']= df['time'].apply(parser)

    df = df[['time','mid_o','mid_c','ask_l','ask_h','bid_l','bid_h']].copy()

    df['PAIR']= pair
            
    df['prev_close']= df.mid_c.shift(1)
    df['prev_ask_h']= df.ask_h.shift(1)
    df['prev_bid_l']= df.bid_l.shift(1)

    df['prev_close_m30']= df.mid_c.shift(30)

    candles = 2880
    #get_res_sup(df,candles)
    #->uncheck if require loading of sure into file
    #pushbullet_message('process completed')
    #print(pair)
    pips_exceed_point=[10]
    pips_tp_point = [100]
    pips_sl_point = [50]

    pips_exceed = [data_library[pair]['pip']*x for x in pips_exceed_point]
    pips_tp = [data_library[pair]['pip']*x for x in pips_tp_point]
    pips_sl = [data_library[pair]['pip']*x for x in pips_sl_point]
    delta_time = [46]
    sure_delta = '24h'

    #make sure load from correct sure levels file as saved in function
    pivot_support_df = pd.read_pickle(f'Backtest sure_levels/{pair}_pivots_support_2880_2015_2017.pkl')
    break_support_df = break_support(df,pips_exceed,pips_tp,delta_time,sure_delta,pips_sl)
    
    df1 = break_support_df
    df1.sort_values(['buy_date'],inplace=True)
    df1.drop_duplicates(subset = ['sure_date'],keep = 'last',inplace=True)
    df1.reset_index(inplace=True,drop=True)
    df1.to_pickle(f'Backtest Results/break_spread_ex10_tp100sl50_{pair}_2015_2017.pkl')

    pivot_resist_df = pd.read_pickle(f'Backtest sure_levels/{pair}_pivots_resist_2880_2015_2017.pkl')
    break_resist_df = break_resist(df,pips_exceed,pips_tp,delta_time,sure_delta,pips_sl)

    df1 = pd.read_pickle(f'Backtest Results/break_spread_ex10_tp100sl50_{pair}_2015_2017.pkl')
    df1=df1.append(break_resist_df)
    df1.sort_values(['buy_date'],inplace=True)
    df1.drop_duplicates(subset = ['sure_date'],keep = 'last',inplace=True)
    df1.reset_index(inplace=True,drop=True)
    df1.to_pickle(f'Backtest Results/break_spread_ex10_tp100sl50_{pair}_2015_2017.pkl')

    pushbullet_message('pair completed')

0.39622641509433965
0.3853211009174312
Message sent
0.39090909090909093
0.3898305084745763
Message sent
0.27522935779816515
0.32432432432432434
Message sent


In [58]:
#input seperate files into collated file
#empty_df = pd.DataFrame()
#empty_df.to_pickle('/Users/User2/Desktop/project/A Sure Bot/Backtest Results/tp100/break_ex10_tp100sl10_2018_0621.pkl')
df3 = pd.read_pickle('/Users/User2/Desktop/project/A Sure Bot/Backtest Results/tp100/Break_ex10_tp100sl50_2015_2017.pkl')
pair_list2 = ['BCO_USD','WTICO_USD', 'XCU_USD']
#'AUD_JPY','AUD_USD','GBP_JPY','EUR_GBP','GBP_USD','USD_CAD','USD_JPY','XAU_USD'
for pair in pair_list2:
    df1 = pd.read_pickle(f'Backtest Results/break_spread_ex10_tp100sl50_{pair}_2015_2017.pkl')
    df3 = df3.append(df1)
    df3.reset_index(inplace=True,drop=True)

df3.to_pickle('/Users/User2/Desktop/project/A Sure Bot/Backtest Results/tp100/Break_ex10_tp100sl50_2015_2017.pkl')
pd.read_pickle('/Users/User2/Desktop/project/A Sure Bot/Backtest Results/tp100/Break_ex10_tp100sl50_2015_2017.pkl')

,buy_date,sell_date,PAIR,sure_level,decision,delta_time,pips_exceed,pips_tp,upper_limit,lower_limit,suc_price,fail_price,pre-entry-candle,break_candle,prev_close_m30,sure_date,sure_detected,sure_delta
0,2015-01-15 17:37:00+00:00,2015-01-16 07:43:00+00:00,GBP_JPY,176.9140,-1,46,10.0,100/50,177.0140,176.8140,NaN,177.35600,176.83100,176.71800,177.25800,2015-01-14 09:00:00+00:00,2015-01-15 09:01:00+00:00,24h
1,2015-01-23 07:53:00+00:00,2015-01-23 08:43:00+00:00,GBP_JPY,177.0660,-1,46,10.0,100/50,177.1660,176.9660,NaN,177.49200,177.08500,176.93000,177.58900,2015-01-21 14:35:00+00:00,2015-01-22 14:36:00+00:00,24h
2,2015-01-25 23:50:00+00:00,2015-01-26 00:02:00+00:00,GBP_JPY,175.8520,-1,46,10.0,100/50,175.9520,175.7520,NaN,176.49400,175.80300,175.73200,176.37200,2015-01-15 22:12:00+00:00,2015-01-18 22:12:00+00:00,24h
3,2015-01-30 07:03:00+00:00,2015-01-30 14:17:00+00:00,GBP_JPY,177.7120,-1,46,10.0,100/50,177.8120,177.6120,176.51500,NaN,177.62800,177.54000,177.75000,2015-01-28 20:47:00+00:00,2015-01-29 20:48:00+00:00,24h
4,2015-02-03 04:00:00+00:00,2015-02-03 10:07:00+00:00,GBP_JPY,176.1390,-1,46,10.0,100/50,176.2390,176.0390,NaN,176.55500,176.10000,175.85400,176.40000,2015-01-30 16:21:00+00:00,2015-02-02 16:21:00+00:00,24h
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3088,2017-11-22 01:21:00+00:00,2017-11-22 01:22:00+00:00,XCU_USD,3.1249,1,46,10.0,100/50,3.1259,3.1239,NaN,3.12060,3.12323,3.12621,3.11835,2017-11-13 15:16:00+00:00,2017-11-14 23:22:00+00:00,24h
3089,2017-11-24 13:05:00+00:00,2017-11-24 13:06:00+00:00,XCU_USD,3.1625,1,46,10.0,100/50,3.1635,3.1615,NaN,3.15501,3.16224,3.16361,3.15637,2017-11-01 05:59:00+00:00,2017-11-02 10:01:00+00:00,24h
3090,2017-12-05 06:40:00+00:00,2017-12-05 06:50:00+00:00,XCU_USD,3.0256,-1,46,10.0,100/50,3.0266,3.0246,NaN,3.02967,3.02485,3.02446,3.04168,2017-11-30 01:59:00+00:00,2017-12-01 07:27:00+00:00,24h
3091,2017-12-15 15:20:00+00:00,2017-12-15 15:21:00+00:00,XCU_USD,3.0948,1,46,10.0,100/50,3.0958,3.0938,NaN,3.08758,3.09268,3.09665,3.07543,2017-12-01 16:11:00+00:00,2017-12-05 00:52:00+00:00,24h


In [61]:
#display results
temp_df = pd.read_pickle('/Users/User2/Desktop/project/A Sure Bot/Backtest Results/tp100/break_ex10_tp100sl50_2018_0621.pkl')

temp_df = temp_df.groupby(['PAIR']).count()

temp_df['suc_%'] = round(temp_df['suc_price']/(temp_df['fail_price']+temp_df['suc_price']),2)

temp_df

,buy_date,sell_date,sure_level,decision,delta_time,pips_exceed,pips_tp,upper_limit,lower_limit,suc_price,fail_price,pre-entry-candle,break_candle,prev_close_m30,sure_date,sure_detected,sure_delta,suc_%
PAIR,,,,,,,,,,,,,,,,,,
AUD_JPY,305,305,305,305,305,305,305,305,305,122,183,305,305,305,305,305,305,0.40
AUD_USD,293,293,293,293,293,293,293,293,293,83,210,293,293,293,293,293,293,0.28
BCO_USD,279,279,279,279,279,279,279,279,279,115,164,279,279,279,279,279,279,0.41
CAD_JPY,313,313,313,313,313,313,313,313,313,101,212,313,313,313,313,313,313,0.32
CORN_USD,13,13,13,13,13,13,13,13,13,4,9,13,13,13,13,13,13,0.31
EUR_CAD,338,338,338,338,338,338,338,338,338,88,250,338,338,338,338,338,338,0.26
EUR_GBP,588,588,588,588,588,588,588,588,588,150,438,588,588,588,588,588,588,0.26
EUR_JPY,307,307,307,307,307,307,307,307,307,83,224,307,307,307,307,307,307,0.27
EUR_NZD,342,342,342,342,342,342,342,342,342,111,231,342,342,342,342,342,342,0.32


In [51]:
pd.read_pickle('/Users/User2/Desktop/project/A Sure Bot/Backtest Results/tp100/break_ex10_tp100sl10_2018_0621.pkl')

,buy_date,sell_date,PAIR,sure_level,decision,delta_time,pips_exceed,pips_tp,upper_limit,lower_limit,suc_price,fail_price,pre-entry-candle,break_candle,prev_close_m30,sure_date,sure_detected,sure_delta
0,2018-01-12 08:57:00+00:00,2018-01-12 17:50:00+00:00,GBP_USD,1.3585,1,46,10.0,100/10,1.3595,1.3575,1.3699,NaN,1.35933,1.35969,1.35564,2018-01-08 01:26:00+00:00,2018-01-09 02:59:00+00:00,24h
1,2018-01-17 00:45:00+00:00,2018-01-17 01:01:00+00:00,GBP_USD,1.3818,1,46,10.0,100/10,1.3828,1.3808,NaN,1.38128,1.38276,1.38305,1.37982,2018-01-15 10:29:00+00:00,2018-01-16 11:28:00+00:00,24h
2,2018-01-22 15:41:00+00:00,2018-01-22 16:54:00+00:00,GBP_USD,1.3942,1,46,10.0,100/10,1.3952,1.3932,NaN,1.39410,1.39481,1.39633,1.39264,2018-01-19 09:37:00+00:00,2018-01-22 09:55:00+00:00,24h
3,2018-01-22 15:41:00+00:00,2018-01-22 17:05:00+00:00,GBP_USD,1.3939,1,46,10.0,100/10,1.3949,1.3929,NaN,1.39379,1.39481,1.39633,1.39264,2018-01-17 18:52:00+00:00,2018-01-18 19:08:00+00:00,24h
4,2018-02-05 20:59:00+00:00,2018-02-05 21:20:00+00:00,GBP_USD,1.3986,-1,46,10.0,100/10,1.3996,1.3976,NaN,1.39861,1.39789,1.39602,1.39962,2018-01-30 08:12:00+00:00,2018-01-31 08:24:00+00:00,24h
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2339,2021-03-25 13:35:00+00:00,2021-04-19 07:49:00+00:00,WHEAT_USD,6.1760,-1,46,10.0,100/50,6.2760,6.0760,NaN,6.57700,6.08400,6.07600,6.13500,2021-03-22 13:40:00+00:00,2021-03-24 11:13:00+00:00,24h
2340,2021-04-09 14:37:00+00:00,2021-04-26 13:35:00+00:00,WHEAT_USD,6.2680,1,46,10.0,100/50,6.3680,6.1680,7.3950,NaN,6.36100,6.36800,6.31200,2021-03-31 16:16:00+00:00,2021-04-05 10:54:00+00:00,24h
2341,2021-04-14 13:33:00+00:00,2021-04-27 10:02:00+00:00,WHEAT_USD,6.3990,1,46,10.0,100/50,6.4990,6.2990,7.5280,NaN,6.44500,6.50000,6.36000,2021-03-23 15:35:00+00:00,2021-03-25 12:11:00+00:00,24h
2342,2021-04-14 13:39:00+00:00,2021-04-27 10:02:00+00:00,WHEAT_USD,6.4220,1,46,10.0,100/50,6.5220,6.3220,7.5280,NaN,6.49200,6.52200,6.36200,2021-04-09 15:44:00+00:00,2021-04-13 10:13:00+00:00,24h
